In [ ]:
nome_radar = 'SaoRoque'

# Declara função

In [ ]:
#-----------------------------------------------------------------------------------
#  Função que plota circulos de distância em geral
#-----------------------------------------------------------------------------------
def evm_plota_aneis_em_geral(aneis, lon_r, lat_r, color, label):

    """
    Retorna círculos de distância centrado no radar

    Parâmetros de entrada:
                           aneis (lista): tamanho do raio do círculo em km
                           lon_r (float): valor da longitude do centro do círculo em graus
                           lat_r (float): valor da latitude do centro do círculo em graus
                           color (str): cor do raio do círculo
                           label (str): legenda

    Parâmetros de saída:
                         latitude e longitude que delimitam os círculos de distância centrado no radar e plota os círculos

    Exemplo:
             evm_plota_aneis_em_geral([100], -45.97279, -23.600795, 'gray', label='Radar: 100 km')
    """

    import geopy                              # Biblioteca para geocodificação
    from geopy import distance                # Função para calculo de distância
    import numpy as np

    origin = geopy.Point(lat_r, lon_r)

    lons = np.zeros((len(aneis), 361))
    lats = np.zeros((len(aneis), 361))
    for i, dis in enumerate(aneis):
        xpts = []
        ypts = []
        for az in range(361):
            destination = distance.distance(kilometers=dis).destination(origin, az)
            lat2, lon2 = destination.latitude, destination.longitude
            xpts.append(lon2)
            ypts.append(lat2)
        lons[i,:] = xpts[:]
        lats[i,:] = ypts[:]

    for i, anel in enumerate(aneis):
        ax.plot(lons[i,:], lats[i,:], color=color, label= label)

#-----------------------------------------------------------------------------------
#        Obtêm as informações do radar a partir do dado bruto (HDF5)
#-----------------------------------------------------------------------------------
def get_info_radar(radar):

    """
    Obtêm as informações dos dados do radar

    Parâmetros de entrada:
                           radar (object): objeto do pyart

    Parâmetros de saída:
                         dic (dicionário): informações do radar, como:
                           date_start(list): ano, mês, dia hora, minuto inicial do scan do radar
                           date_end(list): ano, mês, dia hora, minuto final do scan do radar
                           lon_radar(float): longitude do radar em graus
                           lat_radar(float): latitude do radar em graus
                           alt_radar(float): altitude do radar em metros
                           radar_frequency(float): frequência do radar em Hz
                           radar_beam_width(float): largura do feixe em graus
                           nsweeps(int): quantidade de elevações
                           sweeps(list): ângulos de elevação em graus
                           nrays(list): quantidade de azimutes para cada elevação
                           ngates(int): quantidade de bins
                           radial_resolution(float): resolução radial do radar em metros
                           ray_angle_res(float): resolução azimutal em graus
                           lonmin(float): longitude mínima da matriz de dados
                           lonmax(float): longitude máxima da matriz de dados
                           latmin(float): latitude mínima da matriz de dados
                           latmax(float): latitude máxima da matriz de dados
                           variables(list): variáveis disponíveis

    Exemplo:
             dic = get_info_radar(radar)
    """

    # importa biblioteca de datas
    from netCDF4 import num2date

    # campos disponíveis
    variaveis = radar.fields.keys()

    # latitude e longitude do radar
    lon_radar, lat_radar, alt_radar = radar.longitude['data'][0], radar.latitude['data'][0], radar.altitude['data'][0]

    # informações sobre o tempo
    data_ini = num2date(radar.time['data'][0], radar.time['units'] )
    data_end = num2date(radar.time['data'][-1], radar.time['units'] )

    # extrai o ano, mes dia, hora e minuto inicial da imagem
    anoi, mesi, diai, hori, mini, segi = str(data_ini.year), str(data_ini.month).zfill(2), str(data_ini.day).zfill(2), str(data_ini.hour).zfill(2), str(data_ini.minute).zfill(2), str(data_ini.second).zfill(2)
    anof, mesf, diaf, horf, minf, segf = str(data_end.year), str(data_end.month).zfill(2), str(data_end.day).zfill(2), str(data_end.hour).zfill(2), str(data_end.minute).zfill(2), str(data_end.second).zfill(2)

    # frequência do radar
    radar_frequency = radar.instrument_parameters['frequency']['data'][0]

    # largura do feixe
    radar_beam_width = radar.instrument_parameters['radar_beam_width_h']['data'][0]

    # quantidade TOTAL de bins do feixe
    ngates = radar.ngates

    # quantidade de elevações
    nsweeps = radar.nsweeps

    # quantidade de azimutes por elevação
    total_azimuths = radar.azimuth['data'].shape[0]
    index_azimuths = radar.sweep_start_ray_index['data'].tolist()
    index_azimuths.append(total_azimuths)
    qte_azim = [index_azimuths[i+1] - index_azimuths[i] for i in range(nsweeps)]

    # resolução radial
    radial_resolution = radar.range['meters_between_gates']

    # informação apenas da resolução ângulo azimutal
    ray_angle_res = radar.ray_angle_res['data'][0]

    # ângulos de elevação
    sweeps =  radar.fixed_angle['data']

    # definindo a extensão dos dados em termos de latitude/longitude
    lats = radar.gate_latitude
    lons = radar.gate_longitude
    lonmin = lons['data'].min()
    lonmax = lons['data'].max()
    latmin = lats['data'].min()
    latmax = lats['data'].max()

    # coloca as variáveis num dicionário
    dic = {
            'date_start(year,month,day,hour,minute)': [anoi, mesi, diai, hori, mini],
            'date_end(year,month,day,hour,minute)': [anof, mesf, diaf, horf, minf],
            'lon_radar(degree)': lon_radar,
            'lat_radar(degree)': lat_radar,
            'alt_radar(meters)': alt_radar,
            'radar_frequency(Hz)': radar_frequency,
            'radar_beam_width(degree)': radar_beam_width,
            'nsweeps(#)': nsweeps,
            'sweeps(degree)': list(sweeps),
            'nrays(#)': qte_azim,
            'ngates(#)': ngates,
            'radial_resolution(meters)': radial_resolution,
            'ray_angle_res(degree)': ray_angle_res,
            'lonmin(degree)': lonmin,
            'lonmax(degree)': lonmax,
            'latmin(degree)': latmin,
            'latmax(degree)': latmax,
            'variables': list(variaveis)

           }

    return dic

# Informações do radar

In [ ]:
# leitura do dado 
import pyart                              
filename = 'R13537474_202306151200.hdf5'
radar = pyart.aux_io.read_gamic(f'2023-06-15_sao_roque/{filename}')

# processa a função que extrai as infomações do dado do radar
dic = get_info_radar(radar)

# salva arquivo
with open(f'info_radar_{nome_radar}.csv', 'w') as f:
    for key in dic.keys():
        f.write("%s= %s\n"%(key, dic[key]))

In [ ]:
# mostra as informações do radar na tela
dic

# Plota CAPPI com o pyart para para `apenas 1 horário`

In [ ]:
import time

In [ ]:
%%time
#========================================================================================================================#
#                                             IMPORTA BILIOTECAS
#========================================================================================================================#
import pyart                              
import matplotlib.pyplot as plt           
import cartopy.crs as ccrs                
import warnings
warnings.filterwarnings("ignore")

#========================================================================================================================#
#                                             LEITURA DO DADO DE RADAR
#========================================================================================================================#
# leitura do arquivo
filename = 'R13537474_202306151200.hdf5'
radar = pyart.aux_io.read_gamic(f'2023-06-15_sao_roque/{filename}')

# extrai a latitude, longitude do radar e altitude do radar
lat_radar, lon_radar = radar.latitude['data'][0], radar.longitude['data'][0]
altitude = radar.altitude['data'][0]

# limites dos dados
latmin, latmax = radar.gate_latitude['data'].min(), radar.gate_latitude['data'].max()
lonmin, lonmax = radar.gate_longitude['data'].min(), radar.gate_longitude['data'].max()

# extrai a data do dado do radar
data = pyart.util.datetime_from_grid(radar)

# extração da data da imagem
ano = str(data.year)
mes = str(data.month).zfill(2)
dia = str(data.day).zfill(2)
hor = str(data.hour).zfill(2)
min = str(data.minute).zfill(2)

#========================================================================================================================#
#                                             GERA O CAPPI
#========================================================================================================================#
cappi_altitude = 3000
if (cappi_altitude < altitude):
    print('erro: altura do cappi menor que a altitude do radar')
cappi_altitude = cappi_altitude - altitude

cappi = pyart.map.grid_from_radars(radar, 
                                   grid_shape=(1, 480, 480),
                                   grid_limits=((cappi_altitude, cappi_altitude,),
                                                (-240000., 240000.),
                                                (-240000, 240000.)),
                                   grid_origin = (lat_radar, lon_radar),
                                   roi_func='dist_beam',
                                   zdist_factor=1.4,
                                   h_factor=1.2, 
                                   nb=1.2,#nb menor parece melhor
                                   bsp=0.5, 
                                   weighting_function='barnes2')

#========================================================================================================================#
#                                                     PLOTA FIGURA
#========================================================================================================================#
# define o tamanho da figura
fig = plt.figure(figsize=(10,7))

# define os eixos e projeção da figura
ax = fig.add_subplot(111, projection=ccrs.PlateCarree())

# monta um objeto "display" do Py-ART
display = pyart.graph.GridMapDisplay(cappi)

#corrige o nivel do cappi levando em conta a altitude do radar
display.grid.z['data'] = display.grid.z['data'] + altitude

# plota o cappi
display.plot_grid("corrected_reflectivity",
                  level=0,
                  vmin=0,
                  vmax=70,
                  ax=ax,
                  cmap='pyart_NWSRef',
                  colorbar_label='Refletividade (dBZ)')

# plotas as linhas passando pelo radar
display.plot_crosshairs(lon=lon_radar, lat=lat_radar)

# plota os aneis de distância do radar
evm_plota_aneis_em_geral([100, 250], lon_radar, lat_radar, 'black', label='Radar: 45 km')

# título da figura
plt.title(f'CAPPI {int(display.grid.z["data"][0]) + altitude} m - Refletividade: {data} - {nome_radar}', fontsize=10)

# nome dos eixos x e y
ax.set_xlabel('Longitude ($\degree$)', fontsize=15)
ax.set_ylabel('Latitude ($\degree$)', fontsize=15)

# recorta figura
plt.tight_layout()

# salva figura
plt.savefig(f'{nome_radar}_cappi_{int(display.grid.z["data"][0])}m_z_{ano}{mes}{dia}_{hor}{min}.png', dpi=300)

# Plota CAPPI com o proplot para `apenas 1 horário`

In [ ]:
%%time
#========================================================================================================================#
#                                             IMPORTA BILIOTECAS
#========================================================================================================================#
import pyart                              
import matplotlib.pyplot as plt           
import cartopy.crs as ccrs      
import proplot as pplt
import warnings
warnings.filterwarnings("ignore")

#========================================================================================================================#
#                                             LEITURA DO DADO DE RADAR
#========================================================================================================================#
# leitura do arquivo
filename = 'R13537474_202306151200.hdf5'
radar = pyart.aux_io.read_gamic(f'2023-06-15_sao_roque/{filename}')

# extrai a latitude, longitude do radar e altitude do radar
lat_radar, lon_radar = radar.latitude['data'][0], radar.longitude['data'][0]
altitude = radar.altitude['data'][0]

# limites dos dados
latmin, latmax = radar.gate_latitude['data'].min(), radar.gate_latitude['data'].max()
lonmin, lonmax = radar.gate_longitude['data'].min(), radar.gate_longitude['data'].max()

# extrai a data do dado do radar
data = pyart.util.datetime_from_grid(radar)

# extração da data da imagem
ano = str(data.year)
mes = str(data.month).zfill(2)
dia = str(data.day).zfill(2)
hor = str(data.hour).zfill(2)
min = str(data.minute).zfill(2)

#========================================================================================================================#
#                                             GERA PRECIPITAÇÃO
#========================================================================================================================#
# a) Z-R
chuva_z = pyart.retrieve.est_rain_rate_z(radar, refl_field='corrected_reflectivity')

# adiciona os campos criados na estrutura do objeto "radar"
radar.add_field('rainrate_z', chuva_z)

#========================================================================================================================#
#                                             GERA O CAPPI
#========================================================================================================================#
cappi_altitude = 3000
if (cappi_altitude < altitude):
    print('erro: altura do cappi menor que a altitude do radar')
cappi_altitude = cappi_altitude - altitude

cappi = pyart.map.grid_from_radars(radar, 
                                   grid_shape=(1, 500, 500),
                                   grid_limits=((cappi_altitude, cappi_altitude,),
                                                (-250000., 250000.),
                                                (-250000, 250000.)),
                                   grid_origin = (lat_radar, lon_radar),
                                   roi_func='dist_beam',
                                   zdist_factor=1.4,
                                   h_factor=1.2, 
                                   nb=1.2,#nb menor parece melhor
                                   bsp=0.5, 
                                   weighting_function='barnes2')

# transforma para Dataset do Xarray
ds = cappi.to_xarray()

#========================================================================================================================#
#                                          PLOTA FIGURA: REFLETIVIDADE
#========================================================================================================================#
#--------------------------#
#        mapa radar
#--------------------------#
# define a altura que será plotada
indice_altura = 0 #(=3000m)
altura = int( ds['z'][indice_altura] + altitude) #(=3000)

# cria moldura da figura
fig, ax = pplt.subplots(axheight=5, tight=True, proj='pcarree')

# exatrai os limites dos dados de CAPPI
lonmin, lonmax, latmin, latmax = float(ds['lon'].min()), float(ds['lon'].max()), float(ds['lat'].min()), float(ds['lat'].max())

# formatação dos eixos
ax.format(coast=False, borders=False, innerborders=False,
          labels=True,
          latlines=1, lonlines=1,
          title = f'CAPPI {altura} m: {ano}-{mes}-{dia} às {hor}{min} UTC',
          latlim=(latmin-0.07, latmax+0.06),
          lonlim=(lonmin-0.05, lonmax+0.05),
          small='20px',
          large='15px',
          abc=False)

# plota mapa
map1 = ax.contourf(ds['lon'],
                   ds['lat'],
                   ds['corrected_reflectivity'][0, indice_altura, :, :],
                   cmap='pyart_NWSRef',
                   levels=pplt.arange(0, 70., 2.0),
                   vmin=0, vmax=70)

#--------------------------#
# aneis de distância do radar
#--------------------------#
evm_plota_aneis_em_geral([100], lon_radar, lat_radar, 'gray', label='100 km')
evm_plota_aneis_em_geral([250], lon_radar, lat_radar, 'black', label='250 km')

#--------------------------#
#  barra de cor/legenda
#--------------------------#
fig.colorbar(map1, loc='r', label='Reflectivity (dBZ)', ticks=10, ticklabelsize=15, labelsize=15, width=0.20, space=0.5)

# adiciona legenda
ax.legend(loc='lr', ncols=1, frameon=False, prop={'size':7.5}, markerscale=0.4)

#--------------------------#
#       salva figura
#--------------------------#
fig.save(f'{nome_radar}_cappi_{int(display.grid.z["data"][0])}m_z_{ano}{mes}{dia}_{hor}{min}.png', dpi=300)

#========================================================================================================================#
#                                          PLOTA FIGURA: PRECIPITAÇÃO
#========================================================================================================================#
#--------------------------#
#        mapa radar
#--------------------------#
# define a altura que será plotada
indice_altura = 0 #(=3000m)
altura = int( ds['z'][indice_altura] + altitude) #(=3000)

# cria moldura da figura
fig, ax = pplt.subplots(axheight=5, tight=True, proj='pcarree')

# exatrai os limites dos dados de CAPPI
lonmin, lonmax, latmin, latmax = float(ds['lon'].min()), float(ds['lon'].max()), float(ds['lat'].min()), float(ds['lat'].max())

# formatação dos eixos
ax.format(coast=False, borders=False, innerborders=False,
          labels=True,
          latlines=1, lonlines=1,
          title = f'CAPPI {altura} m: {ano}-{mes}-{dia} às {hor}{min} UTC',
          latlim=(latmin-0.07, latmax+0.06),
          lonlim=(lonmin-0.05, lonmax+0.05),
          small='20px',
          large='15px',
          abc=False)

# plota mapa
map1 = ax.contourf(ds['lon'],
                   ds['lat'],
                   ds['rainrate_z'][0, indice_altura, :, :],
                   cmap='pyart_NWSRef',
                   levels=pplt.arange(0, 80., 5),
                   vmin=0, vmax=80)

#pyart_NWSRef
#pyart_LangRainbow12

#--------------------------#
# aneis de distância do radar
#--------------------------#
evm_plota_aneis_em_geral([100], lon_radar, lat_radar, 'gray', label='100 km')
evm_plota_aneis_em_geral([250], lon_radar, lat_radar, 'black', label='250 km')

#--------------------------#
#  barra de cor/legenda
#--------------------------#
fig.colorbar(map1, loc='r', label='mm/h', ticks=5, ticklabelsize=15, labelsize=15, width=0.20, space=0.5)

# adiciona legenda
ax.legend(loc='lr', ncols=1, frameon=False, prop={'size':7.5}, markerscale=0.4)

#--------------------------#
#       salva figura
#--------------------------#
fig.save(f'{nome_radar}_cappi_{int(display.grid.z["data"][0])}m_rain_{ano}{mes}{dia}_{hor}{min}.png', dpi=300)

# Plota CAPPI com o proplot para `4 horários`

In [ ]:
%%time
#========================================================================================================================#
#                                             IMPORTA BILIOTECAS
#========================================================================================================================#
import pyart                              
import matplotlib.pyplot as plt           
import cartopy.crs as ccrs      
import proplot as pplt
import warnings
warnings.filterwarnings("ignore")

#========================================================================================================================#
#                                             LEITURA DO DADO DE RADAR
#========================================================================================================================#
files = ['R13537474_202306150000.hdf5',
         'R13537474_202306150600.hdf5',
         'R13537474_202306151200.hdf5',
         'R13537474_202306151800.hdf5']

# loop dos arquivos
for file in files:

    print('Processando===>>>', file)

    # leitura do arquivo
    radar = pyart.aux_io.read_gamic(f'2023-06-15_sao_roque/{file}')

    # extrai a latitude, longitude do radar e altitude do radar
    lat_radar, lon_radar = radar.latitude['data'][0], radar.longitude['data'][0]
    altitude = radar.altitude['data'][0]

    # limites dos dados
    latmin, latmax = radar.gate_latitude['data'].min(), radar.gate_latitude['data'].max()
    lonmin, lonmax = radar.gate_longitude['data'].min(), radar.gate_longitude['data'].max()

    # extrai a data do dado do radar
    data = pyart.util.datetime_from_grid(radar)

    # extração da data da imagem
    ano = str(data.year)
    mes = str(data.month).zfill(2)
    dia = str(data.day).zfill(2)
    hor = str(data.hour).zfill(2)
    min = str(data.minute).zfill(2)

    #========================================================================================================================#
    #                                             GERA PRECIPITAÇÃO
    #========================================================================================================================#
    # a) Z-R
    chuva_z = pyart.retrieve.est_rain_rate_z(radar, refl_field='corrected_reflectivity')

    # adiciona os campos criados na estrutura do objeto "radar"
    radar.add_field('rainrate_z', chuva_z)

    #========================================================================================================================#
    #                                             GERA O CAPPI
    #========================================================================================================================#
    cappi_altitude = 3000
    if (cappi_altitude < altitude):
        print('erro: altura do cappi menor que a altitude do radar')
    cappi_altitude = cappi_altitude - altitude

    cappi = pyart.map.grid_from_radars(radar, 
                                       grid_shape=(1, 500, 500),
                                       grid_limits=((cappi_altitude, cappi_altitude,),
                                                (-250000., 250000.),
                                                (-250000, 250000.)),
                                       grid_origin = (lat_radar, lon_radar),
                                       roi_func='dist_beam',
                                       zdist_factor=1.4,
                                       h_factor=1.2, 
                                       nb=1.2,#nb menor parece melhor
                                       bsp=0.5, 
                                       weighting_function='barnes2')

    # transforma para Dataset do Xarray
    ds = cappi.to_xarray()

    #========================================================================================================================#
    #                                          PLOTA FIGURA: REFLETIVIDADE
    #========================================================================================================================#
    #--------------------------#
    #        mapa radar
    #--------------------------#
    # define a altura que será plotada
    indice_altura = 0 #(=3000m)
    altura = int( ds['z'][indice_altura] + altitude) #(=3000)

    # cria moldura da figura
    fig, ax = pplt.subplots(axheight=5, tight=True, proj='pcarree')

    # exatrai os limites dos dados de CAPPI
    lonmin, lonmax, latmin, latmax = float(ds['lon'].min()), float(ds['lon'].max()), float(ds['lat'].min()), float(ds['lat'].max())

    # formatação dos eixos
    ax.format(coast=False, borders=False, innerborders=False,
              labels=True,
              latlines=1, lonlines=1,
              title = f'CAPPI {altura} m: {ano}-{mes}-{dia} at {hor}{min} UTC',
              latlim=(latmin-0.07, latmax+0.06),
              lonlim=(lonmin-0.05, lonmax+0.05),
              small='20px',
              large='15px',
              abc=False)

    # plota mapa
    map1 = ax.contourf(ds['lon'],
                       ds['lat'],
                       ds['corrected_reflectivity'][0, indice_altura, :, :],
                       cmap='pyart_NWSRef',
                       levels=pplt.arange(0, 70., 2.0),
                       vmin=0, vmax=70)

    #--------------------------#
    # aneis de distância do radar
    #--------------------------#
    evm_plota_aneis_em_geral([100], lon_radar, lat_radar, 'gray', label='100 km')
    evm_plota_aneis_em_geral([250], lon_radar, lat_radar, 'black', label='250 km')

    #--------------------------#
    #  barra de cor/legenda
    #--------------------------#
    fig.colorbar(map1, loc='r', label='Reflectivity (dBZ)', ticks=10, ticklabelsize=15, labelsize=15, width=0.20, space=0.5)

    # adiciona legenda
    ax.legend(loc='lr', ncols=1, frameon=False, prop={'size':7.5}, markerscale=0.4)

    #--------------------------#
    #       salva figura
    #--------------------------#
    #fig.save(f'{nome_radar}_cappi_{int(display.grid.z["data"][0])}m_z_{ano}{mes}{dia}_{hor}{min}.png', dpi=300)

    #========================================================================================================================#
    #                                          PLOTA FIGURA: PRECIPITAÇÃO
    #========================================================================================================================#
    #--------------------------#
    #        mapa radar
    #--------------------------#
    # define a altura que será plotada
    indice_altura = 0 #(=3000m)
    altura = int( ds['z'][indice_altura] + altitude) #(=3000)

    # cria moldura da figura
    fig, ax = pplt.subplots(axheight=5, tight=True, proj='pcarree')

    # exatrai os limites dos dados de CAPPI
    lonmin, lonmax, latmin, latmax = float(ds['lon'].min()), float(ds['lon'].max()), float(ds['lat'].min()), float(ds['lat'].max())

    # formatação dos eixos
    ax.format(coast=False, borders=False, innerborders=False,
              labels=True,
              latlines=1, lonlines=1,
              title = f'CAPPI {altura} m: {ano}-{mes}-{dia} at {hor}{min} UTC',
              latlim=(latmin-0.07, latmax+0.06),
              lonlim=(lonmin-0.05, lonmax+0.05),
              small='20px',
              large='15px',
              abc=False)

    # plota mapa
    map1 = ax.contourf(ds['lon'],
                       ds['lat'],
                       ds['rainrate_z'][0, indice_altura, :, :],
                       cmap='pyart_NWSRef',
                       levels=pplt.arange(0.1, 40., 5),
                       vmin=0.1, vmax=40)

    #pyart_NWSRef
    #pyart_LangRainbow12

    #--------------------------#
    # aneis de distância do radar
    #--------------------------#
    evm_plota_aneis_em_geral([100], lon_radar, lat_radar, 'gray', label='100 km')
    evm_plota_aneis_em_geral([250], lon_radar, lat_radar, 'black', label='250 km')

    #--------------------------#
    #  barra de cor/legenda
    #--------------------------#
    fig.colorbar(map1, loc='r', label='Precipitation (mm/h)', ticks=5, ticklabelsize=15, labelsize=15, width=0.20, space=0.5)

    # adiciona legenda
    ax.legend(loc='lr', ncols=1, frameon=False, prop={'size':7.5}, markerscale=0.4)

    #--------------------------#
    #       salva figura
    #--------------------------#
    #fig.save(f'{nome_radar}_cappi_{int(display.grid.z["data"][0])}m_rain_{ano}{mes}{dia}_{hor}{min}.png', dpi=300)

# Acumula a chuva a `cada 3h` e plota CAPPI de chuva acumulada - `USEI ESTA PARTE PARA O ARTIGO`

## 1) Gera e salva cappi no format NETCDF

In [ ]:
%%time
#========================================================================================================================#
#                                             IMPORTA BILIOTECAS
#========================================================================================================================#
import pyart                              
import matplotlib.pyplot as plt           
import cartopy.crs as ccrs      
import proplot as pplt
from datetime import datetime
import xarray as xr
import glob
import numpy as np
import warnings
warnings.filterwarnings("ignore")

#========================================================================================================================#
#                                             LEITURA DO DADO DE RADAR
#========================================================================================================================#
# lista os arquivos
files = sorted(glob.glob(f'2023-06-15_sao_roque/*hdf5'))
files=files[1:]

# loop dos arquivos
for file in files:

    print('Processando===>>>', file)

    #========================================================================================================================#
    #                                             LEITURA DO DADO DO RADAR
    #========================================================================================================================#
    # leitura do arquivo
    radar = pyart.aux_io.read_gamic(file)

    # extrai a latitude, longitude do radar e altitude do radar
    lat_radar, lon_radar = radar.latitude['data'][0], radar.longitude['data'][0]
    altitude = radar.altitude['data'][0]

    # limites dos dados
    latmin, latmax = radar.gate_latitude['data'].min(), radar.gate_latitude['data'].max()
    lonmin, lonmax = radar.gate_longitude['data'].min(), radar.gate_longitude['data'].max()

    # extrai a data do dado do radar
    data = pyart.util.datetime_from_grid(radar)

    # extração da data da imagem
    ano = str(data.year)
    mes = str(data.month).zfill(2)
    dia = str(data.day).zfill(2)
    hor = str(data.hour).zfill(2)
    min = str(data.minute).zfill(2)

    #========================================================================================================================#
    #                                             GERA PRECIPITAÇÃO
    #========================================================================================================================#
    ##### A) Z-R relação clássica a=200 e b=1.6
    chuva_marshall_palmer_classica = pyart.retrieve.est_rain_rate_z(radar, refl_field='corrected_reflectivity')
    
    # adiciona os campos criados na estrutura do objeto "radar"
    radar.add_field('rainrate_MP_classica', chuva_marshall_palmer_classica)
    

    ###### B) Z-R relação alterada a=337 e b=1.38. Valores encontrados para SJC: http://urlib.net/8JMKD3MGP3W34T/4AQSB3S
    refl = radar.fields['corrected_reflectivity']['data']
    z = np.ma.power(10.0, refl/10)
    chuva_marshall_palmer_alterada = (np.ma.power(z/337, 1/1.38))

    # adiciona o campo de precipitação
    radar.add_field_like('corrected_reflectivity', 'rainrate_MP_alterada', chuva_marshall_palmer_alterada)
    radar.fields['rainrate_MP_alterada']['units'] = 'mm/h'
    radar.fields['rainrate_MP_alterada']['standard_name'] = 'Rain_Rate'
    radar.fields['rainrate_MP_alterada']['long_name'] = 'Rain Rate'

    #========================================================================================================================#
    #                                              GERA O CAPPI
    #========================================================================================================================#
    cappi_altitude = 3000
    if (cappi_altitude < altitude):
        print('erro: altura do cappi menor que a altitude do radar')
    cappi_altitude = cappi_altitude - altitude

    cappi = pyart.map.grid_from_radars(radar, 
                                       grid_shape=(1, 500, 500),
                                       grid_limits=((cappi_altitude, cappi_altitude,),
                                                    (-250000., 250000.),
                                                    (-250000, 250000.)),
                                       grid_origin = (lat_radar, lon_radar),
                                       roi_func='dist_beam',
                                       zdist_factor=1.4,
                                       h_factor=1.2, 
                                       nb=1.2,#nb menor parece melhor
                                       bsp=0.5, 
                                       weighting_function='barnes2')
    
    #========================================================================================================================#
    #                                         SALVA CAPPI EM FORMATO NETCDF
    #========================================================================================================================#    
    # transforma para Dataset do Xarray
    cappi_dataset = cappi.to_xarray()

    # transforma o dataset para array: (y, x)
    MP1_values = cappi_dataset['rainrate_MP_classica'][0,0,:,:].values
    MP2_values = cappi_dataset['rainrate_MP_alterada'][0,0,:,:].values    
    dbz_values = cappi_dataset['corrected_reflectivity'][0,0,:,:].values 

    # define encoding para compactação dos dados
    encoding = {
                'rainrate_MP_classica': {'zlib': True},
                'rainrate_MP_alterada': {'zlib': True},
                'corrected_reflectivity': {'zlib': True},
               }

    # define as variáveis
    data_vars = {
                 'rainrate_MP_classica': (('lat', 'lon'), MP1_values, {'units': 'mm/h', 'long_name': 'Taxa de Precipitação', '_FillValue': 0}),  
                 'rainrate_MP_alterada': (('lat', 'lon'), MP2_values, {'units': 'mm/h', 'long_name': 'Taxa de Precipitação', '_FillValue': 0}),         
                 'corrected_reflectivity': (('lat', 'lon'), dbz_values, {'units': 'dBZ', 'long_name': 'Horizontal Reflectivity', '_FillValue': 0}),
                }
  
    # coordenadas 
    #coords = {'lat': cappi_dataset['lat'][:,0].values, 'lon': cappi_dataset['lon'][:,0].values}
    coords = {'lat': cappi_dataset['lat'][:,0].values, 'lon': cappi_dataset['lon'][0, :].values}

    # ARRUMEI
    # mudei de ** 'lon': cappi_dataset['lon'][:,0].values **  PARA ** 'lon': cappi_dataset['lon'][0,:].values **

    # atributos 
    descricao = """Constant Altitude Plan Position Indicator (CAPPI) de 3 km de altitude
    com resolução espacial horizontal de 1 km para o radar de São Roque (SP)"""

    attrs = {'description': descricao,                             
             'creation_data': str(datetime.now()), 
             'author':'Enrique V. Mattos',
             'email': 'enrique@unifei.edu.br'}

    # gera Dataset
    dataset = xr.Dataset(data_vars = data_vars, coords = coords, attrs = attrs)

    # salva arquivo netcdf
    dataset.to_netcdf(f'01_output_arquivos_netcdf_z_rain/cappi_3000m_z_rain_saoroque_{ano}{mes}{dia}_{hor}{min}.nc',  encoding=encoding)

In [ ]:
# exemplo do arquivo NETCDF gerado
netcdf = xr.open_dataset('01_output_arquivos_netcdf_z_rain/cappi_3000m_z_rain_saoroque_20230615_0000.nc')
netcdf

In [ ]:
cappi_dataset['rainrate_MP_classica'].max()

In [ ]:
cappi_dataset['rainrate_MP_alterada'].max()

In [ ]:
netcdf['rainrate_MP_classica'][:,:].plot(figsize=(9,7), cmap='pyart_NWSRef', vmin=0, vmax=40)

In [ ]:
netcdf['rainrate_MP_alterada'][:,:].plot(figsize=(9,7), cmap='pyart_NWSRef', vmin=0, vmax=40)

In [ ]:
netcdf['corrected_reflectivity'][:,:].plot(figsize=(9,7), cmap='pyart_NWSRef', vmin=0, vmax=70)

## 2) Plota Z e precipitação dos arquivos NETCDF

In [ ]:
ds = xr.open_dataset('01_output_arquivos_netcdf_z_rain/cappi_3000m_z_rain_saoroque_20230615_1200.nc')
ds

In [ ]:
%%time
# importa bibliotecas
import pyart                              
import matplotlib.pyplot as plt           
import cartopy.crs as ccrs      
import proplot as pplt
import glob
import os
import xarray as xr
import warnings
warnings.filterwarnings("ignore")

# localização do radar
lon_radar, lat_radar = -47.0943, -23.602

# lista dos arquivos
files = sorted(glob.glob(f'01_output_arquivos_netcdf_z_rain/cappi_3000m_z_rain_saoroque_20230615*.nc')) # 144 arquivos

# loop dos arquivos
for file in files:

    print('Processando file ===>>>', file)

    # nome do arquivo
    basename = os.path.basename(os.path.splitext(file)[0]) # cappi_3000m_z_rain_saoroque_20230615_0000.nc

    # extrai a data da imagem
    ano, mes, dia, hor, min = basename[28:32], basename[32:34], basename[34:36], basename[37:39], basename[39:41]

    # lendo arquivo
    ds = xr.open_dataset(file)

    #========================================================================================================================#
    #                                          PLOTA FIGURA: REFLETIVIDADE
    #========================================================================================================================#
    #--------------------------#
    #        mapa radar
    #--------------------------#
    # cria moldura da figura
    fig, ax = pplt.subplots(axheight=5, tight=True, proj='pcarree')

    # exatrai os limites dos dados de CAPPI
    lonmin, lonmax, latmin, latmax = float(ds['lon'].min()), float(ds['lon'].max()), float(ds['lat'].min()), float(ds['lat'].max())

    # formatação dos eixos
    ax.format(coast=False, borders=False, innerborders=False,
              labels=True,
              latlines=1, lonlines=1,
              title = f'CAPPI 3000 m: {ano}-{mes}-{dia} at {hor}{min} UTC',
              latlim=(latmin-0.07, latmax+0.06),
              lonlim=(lonmin-0.05, lonmax+0.05),
              small='20px',
              large='15px',
              abc=False)

    # plota mapa
    map1 = ax.contourf(ds['lon'],
                       ds['lat'],
                       ds['corrected_reflectivity'][:,:],
                       cmap='pyart_NWSRef',
                       levels=pplt.arange(0, 70., 2.0),
                       vmin=0, vmax=70)

    #--------------------------#
    # aneis de distância do radar
    #--------------------------#
    evm_plota_aneis_em_geral([100], lon_radar, lat_radar, 'gray', label='100 km')
    evm_plota_aneis_em_geral([250], lon_radar, lat_radar, 'black', label='250 km')

    #--------------------------#
    #  barra de cor/legenda
    #--------------------------#
    fig.colorbar(map1, loc='r', label='Reflectivity (dBZ)', ticks=10, ticklabelsize=15, labelsize=15, width=0.20, space=0.5)

    # adiciona legenda
    ax.legend(loc='lr', ncols=1, frameon=False, prop={'size':7.5}, markerscale=0.4)

    #--------------------------#
    #       salva figura
    #--------------------------#
    fig.save(f'02_output_figuras_z/{nome_radar}_cappi_3000m_z_{ano}{mes}{dia}_{hor}{min}.png', dpi=300)

    #========================================================================================================================#
    #                                          PLOTA FIGURA: PRECIPITAÇÃO
    #========================================================================================================================#
    #--------------------------#
    #        mapa radar
    #--------------------------#
    # cria moldura da figura
    fig, ax = pplt.subplots(axheight=5, tight=True, proj='pcarree')

    # exatrai os limites dos dados de CAPPI
    lonmin, lonmax, latmin, latmax = float(ds['lon'].min()), float(ds['lon'].max()), float(ds['lat'].min()), float(ds['lat'].max())

    # formatação dos eixos
    ax.format(coast=False, borders=False, innerborders=False,
              labels=True,
              latlines=1, lonlines=1,
              title = f'CAPPI 3000 m: {ano}-{mes}-{dia} at {hor}{min} UTC',
              latlim=(latmin-0.07, latmax+0.06),
              lonlim=(lonmin-0.05, lonmax+0.05),
              small='20px',
              large='15px',
              abc=False)

    # plota mapa
    map1 = ax.contourf(ds['lon'],
                       ds['lat'],
                       ds['rainrate_MP_alterada'][:,:],
                       cmap='pyart_NWSRef',
                       levels=pplt.arange(0.1, 40., 5),
                       vmin=0.1, vmax=40)
    #pyart_NWSRef
    #pyart_LangRainbow12

    #--------------------------#
    # aneis de distância do radar
    #--------------------------#
    evm_plota_aneis_em_geral([100], lon_radar, lat_radar, 'gray', label='100 km')
    evm_plota_aneis_em_geral([250], lon_radar, lat_radar, 'black', label='250 km')

    #--------------------------#
    #  barra de cor/legenda
    #--------------------------#
    fig.colorbar(map1, loc='r', label='Precipitation (mm/h)', ticks=5, ticklabelsize=15, labelsize=15, width=0.20, space=0.5)

    # adiciona legenda
    ax.legend(loc='lr', ncols=1, frameon=False, prop={'size':7.5}, markerscale=0.4)

    #--------------------------#
    #       salva figura
    #--------------------------#
    fig.save(f'03_output_figuras_rain/{nome_radar}_cappi_3000m_rain_{ano}{mes}{dia}_{hor}{min}.png', dpi=300)

## 3) Acumula chuva 3h 

In [ ]:
ds = xr.open_dataset('01_output_arquivos_netcdf_z_rain/cappi_3000m_z_rain_saoroque_20230615_1200.nc')
ds

In [ ]:
%%time
#========================================================================================================================#
#                                              IMPORTAÇÃO DAS BIBLIOTECAS
#========================================================================================================================#
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import pyart
import pandas as pd
import numpy as np
import glob
from datetime import datetime
import xarray as xr
import warnings
warnings.filterwarnings("ignore")

#========================================================================================================================#
#                                               MONTA A LISTA DOS ARQUIVOS
#========================================================================================================================#
# horários
horarios = ['06', '12', '18']

# monta a lista dos arquivos
files1 = sorted(glob.glob(f'01_output_arquivos_netcdf_z_rain/cappi_3000m_z_rain_saoroque_20230615_03*.nc')) 
files2 = sorted(glob.glob(f'01_output_arquivos_netcdf_z_rain/cappi_3000m_z_rain_saoroque_20230615_04*.nc')) 
files3 = sorted(glob.glob(f'01_output_arquivos_netcdf_z_rain/cappi_3000m_z_rain_saoroque_20230615_05*.nc')) 
files_06utc = files1+files2+files3

files1 = sorted(glob.glob(f'01_output_arquivos_netcdf_z_rain/cappi_3000m_z_rain_saoroque_20230615_09*.nc')) 
files2 = sorted(glob.glob(f'01_output_arquivos_netcdf_z_rain/cappi_3000m_z_rain_saoroque_20230615_10*.nc')) 
files3 = sorted(glob.glob(f'01_output_arquivos_netcdf_z_rain/cappi_3000m_z_rain_saoroque_20230615_11*.nc')) 
files_12utc = files1+files2+files3

files1 = sorted(glob.glob(f'01_output_arquivos_netcdf_z_rain/cappi_3000m_z_rain_saoroque_20230615_15*.nc')) 
files2 = sorted(glob.glob(f'01_output_arquivos_netcdf_z_rain/cappi_3000m_z_rain_saoroque_20230615_16*.nc')) 
files3 = sorted(glob.glob(f'01_output_arquivos_netcdf_z_rain/cappi_3000m_z_rain_saoroque_20230615_17*.nc')) 
files_18utc = files1+files2+files3

#========================================================================================================================#
#                                                 PROCESSAMENTO
#========================================================================================================================#
# taxa de conversão de 1 hora para 10min 
fator = 10./60.

# loop das datas
for horario in horarios:

    print('horario===>>', horario)
    
    # inicializa variável de soma da chuva
    chuva_total = np.zeros([500,500])

    # define as listas de arquivos
    if horario=='06':
        files = files_06utc
    elif horario=='12': 
        files = files_12utc
    else: 
        files = files_18utc
        
    # loop dos arquivos daquela data
    for file in files:

        print('PROCESSANDO ===>>', file)

        # leitura do arquivo netcdf
        ds = xr.open_dataset(file)

        # transforma "NaN" para "0.0"
        chuva_atual = ds['rainrate_MP_alterada'][:,:]
        chuva_atual = np.where(~np.isnan(chuva_atual), chuva_atual, 0.0)

        # soma total de chuva
        chuva_total = chuva_total + (chuva_atual*fator)

    # Salva arquivo em formato NETCDF
    # define encoding para compactação dos dados
    encoding = {'rain_acum_3h': {'zlib': True}}

    # define as variáveis
    data_vars = {'rain_acum_3h': (('lat', 'lon'), chuva_total, {'units': 'mm/3h', 'long_name': 'Acumlado de Precipitação no Intervalo de 3h', '_FillValue': 0})}
  
    # coordenadas 
    coords = {'lat': ds['lat'].values, 'lon': ds['lon'].values}

    # atributos  
    descricao = """Constant Altitude Plan Position Indicator (CAPPI) de precipitação em 3 km de altitude com resolução espacial horizontal de 1 km para o radar de São Roque (SP)
    localizado em lon_radar=-47.0943 e lat_radar=-23.602. Foi utilizado a relação de Marshall-Palmer adaptada para cidade de São José dos Campos a=337 e b=1.38: http://urlib.net/8JMKD3MGP3W34T/4AQSB3S"""
    
    attrs = {'description': descricao,                             
             'creation_data': str(datetime.now()), 
             'author':'Enrique V. Mattos',
             'email': 'enrique@unifei.edu.br'}

    # gera Dataset
    dataset = xr.Dataset(data_vars = data_vars, coords = coords, attrs = attrs)

    # salva arquivo netcdf
    dataset.to_netcdf(f'04_output_netcdf_e_figuras_rain_acumulada_3h/rainacum_3h_saoroque_20230615_{horario}utc.nc',  encoding=encoding)    

## 4) Plota cappi de chuva acumulada em 3h

In [ ]:
ds = xr.open_dataset('04_output_netcdf_e_figuras_rain_acumulada_3h/rainacum_3h_saoroque_20230615_12utc.nc')
ds

In [ ]:
%%time
# importa bibliotecas
import pyart                              
import matplotlib.pyplot as plt           
import cartopy.crs as ccrs      
import proplot as pplt
import glob
import os
import xarray as xr
import warnings
warnings.filterwarnings("ignore")

# localização do radar
lon_radar, lat_radar = -47.0943, -23.602

# lista dos arquivos
files = sorted(glob.glob(f'04_output_netcdf_e_figuras_rain_acumulada_3h/*.nc')) # 44 arquivos

# loop dos arquivos
for file in files:

    print('Processando file ===>>>', file)

    # nome do arquivo
    basename = os.path.basename(os.path.splitext(file)[0]) # rainacum_3h_saoroque_20230615_06utc.nc

    # extrai a data da imagem
    ano, mes, dia, hor = basename[21:25], basename[25:27], basename[27:29], basename[30:32]
    
    # lendo arquivo
    ds = xr.open_dataset(file)

    #========================================================================================================================#
    #                                          PLOTA FIGURA: PRECIPITAÇÃO
    #========================================================================================================================#
    #--------------------------#
    #        mapa radar
    #--------------------------#
    # cria moldura da figura
    fig, ax = pplt.subplots(axheight=5, tight=True, proj='pcarree')

    # exatrai os limites dos dados de CAPPI
    lonmin, lonmax, latmin, latmax = float(ds['lon'].min()), float(ds['lon'].max()), float(ds['lat'].min()), float(ds['lat'].max())

    # formatação dos eixos
    if hor == '06': 
        name = '03-06'
    elif hor == '12': 
        name = '09-12'
    else:
        name = '15-18'
    
    ax.format(coast=False, borders=False, innerborders=False,
              labels=True,
              latlines=1, lonlines=1,
              title = f'Precipitação Acumulada: {name} UTC',
              latlim=(latmin-0.07, latmax+0.06),
              lonlim=(lonmin-0.05, lonmax+0.05),
              small='20px',
              large='15px',
              abc=False)

    # plota mapa
    map1 = ax.contourf(ds['lon'],
                       ds['lat'],
                       ds['rain_acum_3h'][:,:],
                       cmap='pyart_NWSRef',
                       levels=pplt.arange(0.1, 40., 5),
                       vmin=0.1, vmax=40)
    #pyart_NWSRef
    #pyart_LangRainbow12

    #--------------------------#
    # aneis de distância do radar
    #--------------------------#
    evm_plota_aneis_em_geral([100], lon_radar, lat_radar, 'gray', label='100 km')
    evm_plota_aneis_em_geral([250], lon_radar, lat_radar, 'black', label='250 km')

    #--------------------------#
    #  barra de cor/legenda
    #--------------------------#
    fig.colorbar(map1, loc='r', label='Precipitation (mm/3h)', ticks=5, ticklabelsize=15, labelsize=15, width=0.20, space=0.5)

    # adiciona legenda
    ax.legend(loc='lr', ncols=1, frameon=False, prop={'size':7.5}, markerscale=0.4)

    #--------------------------#
    #       salva figura
    #--------------------------#
    fig.save(f'04_output_netcdf_e_figuras_rain_acumulada_3h/{nome_radar}_rain_3h_{hor}utc.png', dpi=300)

In [ ]:
# exemplo do arquivo NETCDF gerado
netcdf = xr.open_dataset('04_output_netcdf_e_figuras_rain_acumulada_3h/rainacum_3h_saoroque_20230615_12utc.nc')
netcdf

In [ ]:
# plota mapa de Z
netcdf['rain_acum_3h'][:,:].plot(figsize=(9,7), cmap='pyart_NWSRef', vmin=0, vmax=40)